# Vinbigdata Chest X-ray Object Detection & Multi-Class Classification 2021

>  ### Problem Statement: This is an object detection and multi-label classification problem

#### Task 

In this competition, I am trying to localize and classify **14 types of thoracic abnormalities** from chest radiographs (X-rays).

#### Dataset information

The dataset consisting of 18,000 (X-ray) scans that have been annotated by experienced radiologists.

The images are in DICOM format, which means they contain 'meta data' that might be useful for visualizing and classifying.

All images were labeled by a panel of experienced radiologists for the presence of 14 critical radiographic findings as listed below:

- 0 - Aortic enlargement
- 1 - Atelectasis
- 2 - Calcification
- 3 - Cardiomegaly
- 4 - Consolidation
- 5 - ILD
- 6 - Infiltration
- 7 - Lung Opacity
- 8 - Nodule/Mass
- 9 - Other lesion
- 10 - Pleural effusion
- 11 - Pleural thickening
- 12 - Pneumothorax
- 13 - Pulmonary fibrosis
- 14 - No findings

### Machine Learning Model

I am buildig objection detection model with **YOLOv4** to train with 15,000 independently-labeled images.

I'd be evaluating it on a given test set of 3,000 images.

For each test image, I will be predicting a bounding box and class for all findings. 

If the prediction is no findings, I should create a prediction of "14 1 0 0 1 1" (14 is the class ID for no finding, and this provides a one-pixel bounding box with a confidence of 1.0).

> ## Let's first start with the preprocessing of the data and do some insightful exploratory analysis!

In [ ]:
# import libraries required
import os,sys,PIL,pydicom
from PIL import Image
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from seaborn import countplot
from matplotlib.pyplot import suptitle
from pydicom import dcmread
from pydicom.data import get_testdata_file
from pathlib import Path
from pydicom.datadict import DicomDictionary, keyword_dict
from pydicom.data import get_testdata_file
import tempfile

## Preprocessing

### Number of DICOM image files 

In [ ]:
# path to train and test image dicom files
source_dir_train = Path('../input/vinbigdata-chest-xray-abnormalities-detection/train/')
source_dir_test = Path('../input/vinbigdata-chest-xray-abnormalities-detection/test/')
# lists of dicom files
imgtrain_files  = os.listdir(source_dir_train)
imgtest_files =os.listdir(source_dir_test)
# number of files
print("Number of training image files with meta data:",len(imgtrain_files))
print("Number of test image files with meta data    :",len(imgtest_files))

### Load train data

In [ ]:
train_data = pd.read_csv("../input/vinbigdata-chest-xray-abnormalities-detection/train.csv")
pd.set_option('display.max_colwidth',None)
train_data.head()

#### NA values replaced  with zero in last 4 columns

In [ ]:
train_data.fillna(0,inplace=True)
train_data.head()

In [ ]:
print("Number of Unique Image IDs      :",len(train_data['image_id'].unique()))
print("Number of Classes               :",len(train_data['class_name'].unique()))
print("Number of Unique Radiologist IDs:",len(train_data['rad_id'].unique()))

## Data Structure of Dicom file

### Data Element Tag

(Group Number,Element Number) - An ordered pair of 16-bit unsigned integers 

### Value Representation (VR)

The VR for Data Element Tag - Two single byte characters - encoded using only upper case letters from the DICOM default character set

### Value Length

16 or 32-bit unsigned integer containing the Explicit Length of the Value Field as the number of bytes (even) that make up the Value

OR

32-bit Length Field set to Undefined Length (FFFFFFFFH). 

### Value Field

An even number of bytes containing the Value(s) of the Data Element.

The data type of Value(s) stored in this field is specified by the Data Element's VR. The VR for a given Data Element Tag can be determined using the Data Dictionary. The Value Multiplicity specifies how many Values with this VR can be placed in the Value Field.


[Read Michele's Blog to Understand DICOM](https://mscipio.github.io/post/read_dicom_files_in_python/)

### Read DICOM File

In [ ]:
file_path = source_dir_train
file_name = imgtrain_files[9]
ds = pydicom.dcmread(os.path.join(file_path,file_name))
ds

### Read DICOM Meta data 

In [ ]:
print(ds.file_meta)

### Read DICOM Dictionary

In [ ]:
ds.dir()

### Parsing the data elements from the DICOM file

In [ ]:
def myprint(dataset, indent=0):
    dont_print = ['Pixel Data', 'File Meta Information Version']
    indent_string = "   " * indent
    next_indent_string = "   " * (indent + 1)
    
    for data_element in dataset:
        if data_element.VR == "SQ":   # a sequence
            print(indent_string, data_element.name)
            for sequence_item in data_element.value:
                myprint(sequence_item, indent + 1)
                print(next_indent_string + "---------")
        else:
            if data_element.name in dont_print:
                print("""<item not printed -- in the "don't print" list>""")
            else:
                repr_value = repr(data_element.value)
                if len(repr_value) > 50:
                    repr_value = repr_value[:50] + "..."
                print("{0:s} {1:s} = {2:s}".format(indent_string,
                                                   data_element.name,
                                                   repr_value))

### Parsing the data elements from DICOM file

In [ ]:
file_path = source_dir_train
file_name = imgtrain_files[9]
ds = pydicom.dcmread(os.path.join(file_path,file_name))
myprint(ds)

In [ ]:
#Function to take care of teh translation and windowing

def window_image(img, window_center,window_width, intercept, slope, rescale=True):
    img = (img*slope +intercept) #for translation adjustments given in the dicom file. 
    img_min = window_center - window_width//2 #minimum HU level
    img_max = window_center + window_width//2 #maximum HU level
    img[img<img_min] = img_min #set img_min for all HU levels less than minimum HU level
    img[img>img_max] = img_max #set img_max for all HU levels higher than maximum HU level
    if rescale: 
        img = (img - img_min) / (img_max - img_min)*255.0 
    return img
    
def get_first_of_dicom_field_as_int(x):
    #get x[0] as in int is x is a 'pydicom.multival.MultiValue', otherwise get int(x)
    if type(x) == dcm.multival.MultiValue: return int(x[0])
    else: return int(x)
    
def get_windowing(data):
    dicom_fields = [data[('0028','1050')].value, #window center
                    data[('0028','1051')].value, #window width
                    data[('0028','1052')].value, #intercept
                    data[('0028','1053')].value] #slope
    return [get_first_of_dicom_field_as_int(x) for x in dicom_fields]

In [ ]:
def view_images(files, title = '', aug = None, windowing = True):
    width = 2
    height = 2
    fig, axs = plt.subplots(height, width, figsize=(15,15))
    
    for im in range(0, height * width):
        data = dcm.dcmread(files[im])
        image = data.pixel_array
        window_center , window_width, intercept, slope = get_windowing(data)
        if windowing:
            output = window_image(image, window_center, window_width, intercept, slope, rescale = False)
        else:
            output = image
        i = im // width
        j = im % width
        axs[i,j].imshow(output, cmap=plt.cm.gray) 
        axs[i,j].axis('off')
        
    plt.suptitle(title)
    plt.show()

### X-ray image

In [ ]:
# Function to plot the X-ray images from dicom files
file_path = source_dir_train
file_name = imgtrain_files[9]

def show_image(PATH,NAME):
    ds = pydicom.dcmread(os.path.join(PATH,NAME))
    plt.imshow(ds.pixel_array,cmap=plt.cm.gray)
    plt.show()
    
show_image(file_path,file_name)

### Down sampling the image

In [ ]:
# function to down sample the image and change BitsStored to 16

def down_sample(PATH,NAME):
    ds = pydicom.dcmread(os.path.join(PATH,NAME))
    ds.BitsStored= 16 # BitsStored replace to 16
    data = ds.pixel_array
    data_downsampling = data[::16, ::16]
    # copy the data back to the original data set
    ds.PixelData = data_downsampling.tobytes()
    # update the information regarding the shape of the data array
    ds.Rows, ds.Columns = data_downsampling.shape
    return ds
    
file_path = source_dir_train
file_name = imgtrain_files[9]
down_sample(file_path,file_name)

### Reading all image files and saving to list

In [ ]:
imgtrain_files = os.listdir(Path('../input/vinbigdata-chest-xray-abnormalities-detection/train/'))
imgtest_files = os.listdir(Path('../input/vinbigdata-chest-xray-abnormalities-detection/test/'))

file_path = source_dir_train
file_path1 = source_dir_test

for file_name in imgtrain_files:
    downsampled_tr_files=[]
    downsampled_tr_files.append(down_sample(file_path,file_name))
    
for file_name in imgtest_files:
    downsampled_test_files=[]
    downsampled_test_files.append(down_sample(file_path1,file_name))

In [ ]:
downsampled_tr_files[0]

## Exploratory Data Analysis

In [ ]:
print("Number of Image ID",len(train_data['image_id'].unique()))
print("Number of Class",len(train_data['class_name'].unique()))
print("Number of Radiologist ID",len(train_data['rad_id'].unique()))

### Names of Class ID  & Class

In [ ]:
class_names = train_data.class_name.unique()
class_ID = train_data.class_id.unique()
Radiologist_ID = train_data.rad_id.unique()

for id, name in sorted(zip(class_ID,class_names)):
    print("ClassID =",id,"->",name)

### Distribution of Class

In [ ]:
plt.figure(figsize=(20,5))
sns.kdeplot(data=train_data['x_min'].value_counts())
plt.suptitle("Count values of findings\n")
plt.xlabel("Number of findings\n")
plt.show()
plt.figure(figsize=(20,5))
sns.kdeplot(data=train_data['x_max'].value_counts())
plt.suptitle("Count values of findings\n")
plt.xlabel("Number of findings\n")
plt.show()

In [ ]:
plt.figure(figsize=(20,5))
sns.kdeplot(train_data['x_min'].value_counts())
plt.suptitle("Count valuys of findings\n")
plt.xlabel("Number of findings\n")
plt.figure(figsize=(20,5))
sns.kdeplot(train_data['y_max'].value_counts())
plt.suptitle("Count values of findings\n")
plt.xlabel("Number of findings\n")
plt.show()

In [ ]:
plt.figure(figsize=(30,10))
sns.countplot(x="class_name",data=train_data)
plt.suptitle("Count values of findings")
plt.xlabel("Class\n")
plt.show()

In [ ]:
import gc
gc.collect()

In [ ]:
plt.figure(figsize=(10,10))
sns.countplot(y="rad_id",data=train_data)
plt.suptitle("Count values of findings\n")
plt.xlabel("Number of findings\n")
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
plt.scatter(train_data.x_min,train_data.x_max,alpha=0.5)
plt.show()
plt.figure(figsize=(10,10))
plt.scatter(train_data.y_min,train_data.y_max,alpha=0.5)
plt.show()

In [ ]:
plt.figure(figsize=(30,10))
sns.boxplot(x=train_data.class_name,y=train_data.x_max)
plt.show()
plt.figure(figsize=(30,10))
sns.boxplot(x=train_data.class_name,y=train_data.y_max)
plt.show()

### count plot

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_context('paper',font_scale=1.5)
plt.figure(figsize=(20,15))
sns.countplot(y=ifood2.state,hue=ifood2['diet'])
plt.ylabel('States\n')
plt.xlabel('counts\n')
plt.title("Number of Food Items from Each State - Veg & Non-Veg\n")
plt.show()